In [26]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

import os
import speech_recognition as sr
from pydub import AudioSegment
from enum import Enum
import tempfile
import requests
import time


In [11]:
if __name__ == '__main__':
    email = 'muhammad.farras.reswara-2020@ftmm.unair.ac.id'
    password = 'farras210102'
    
    options = webdriver.ChromeOptions()
    options.add_argument('proxy-server=106.122.8.54:3128')
    options.add_argument('--user-data-dir=C:\\Users\\farras\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

    driver = uc.Chrome(chrome_options=options)
    driver.get('https://www.youtube.com/c/TechWithTim/about')


In [18]:
options = uc.ChromeOptions()

# options.add_argument('proxy-server=106.122.8.54:3128')
options.add_argument('--user-data-dir=C:\\Users\\farras\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Selenium')

browser = uc.Chrome(options=options, driver_executable_path='C:/SAINS DATA/chromedriver.exe')

browser.get('https://www.youtube.com/c/WorstPremadeEver/about')



In [28]:
class status(Enum):
    SUCCESS = 0
    UNKNOWN = 1
    RATELIMITED = 2
    TIMEOUT = 3

class NotExistent(Exception):
    """
    This exception is used internally
    """
    err = None

    def __init__(self, error, *args: object) -> None:
        super().__init__(*args)

        self.err = error

def __cleanup(files: list):
    for x in files:
        if os.path.exists(x):
            os.remove(x)

ret = None
tmp_dir = tempfile.gettempdir()
mp3_file = os.path.join(tmp_dir, "_tmp.mp3")
wav_file = os.path.join(tmp_dir, "_tmp.wav")
tmp_files = [mp3_file, wav_file]

In [29]:
# scroll
element = driver.find_element(By.CSS_SELECTOR, '#details-container > table > tbody > tr:nth-child(1) > td:nth-child(3) > ytd-button-renderer')
driver.execute_script("arguments[0].scrollIntoView();", element)
time.sleep(1)

# Press view email address button
driver.find_element(By.CSS_SELECTOR, 'td.style-scope:nth-child(3) > ytd-button-renderer:nth-child(1) > a:nth-child(1)').click()

# Wait until captcha iframe loaded
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#recaptcha")))

# Filter through all the iframes on the page and find the one that corresponds to the captcha
iframes = driver.find_elements(By.TAG_NAME, 'iframe')
for iframe in iframes:
    if iframe.get_attribute("src").startswith("https://www.google.com/recaptcha/api2/anchor"):
        captcha = iframe

# Switch current context
driver.switch_to.frame(captcha)

# Click the checkbox
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "recaptcha-checkbox-border")))
driver.find_element(By.CLASS_NAME,"recaptcha-checkbox-border").click()

# CASE 1: captcha image showed
# Back to main iframe
driver.switch_to.default_content()
# print('BACK')

# Image iframe
iframesCount = driver.find_elements(By.TAG_NAME, 'iframe')

captchaState = 0

for iframe2 in iframesCount:
    # CASE 1: image selection appeared
    if iframe2.get_attribute("src").startswith("https://www.google.com/recaptcha/api2/bframe"):
        captcha2 = iframe2
        # print(captcha2)
        # print('Found the iframe!')

        driver.switch_to.frame(captcha2)
        # print('inner iframe')

        # Get the audio challenge instead
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#recaptcha-audio-button")))
        driver.find_element(By.CSS_SELECTOR, '#recaptcha-audio-button').click()

        # Download & convert the file
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body > div > div > div.rc-audiochallenge-tdownload > a")))
        download_link = driver.find_element(By.CSS_SELECTOR, "body > div > div > div.rc-audiochallenge-tdownload > a")

        if not download_link:
            raise NotExistent(status.RATELIMITED)

        with open(mp3_file, "wb") as f:
            link = download_link.get_attribute("href")
            r = requests.get(link, allow_redirects=True)
            f.write(r.content)
            f.close()

        # Convert to wav here
        AudioSegment.from_mp3(mp3_file).export(wav_file, format="wav")

        # Using google's own api against them
        recognizer = sr.Recognizer()

        with sr.AudioFile(wav_file) as source:
            recorded_audio = recognizer.listen(source)
            text = recognizer.recognize_google(recorded_audio)
        print('audio recognized!')

        # Type out the answer
        driver.find_element(By.CSS_SELECTOR, '#audio-response').send_keys(text)

        # Click the "Verify" button to complete
        driver.find_element(By.CSS_SELECTOR, '#recaptcha-verify-button').click()
        time.sleep(1)

        # Press the Submit button
        driver.switch_to.default_content()
        driver.find_element(By.CSS_SELECTOR, '#submit-btn').click()

        __cleanup(tmp_files)

        captchaState = 1
            
#CASE 2: captcha completed instantly
if captchaState == 0:
    # Switch current context
    driver.switch_to.frame(captcha)

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#recaptcha-anchor > div.recaptcha-checkbox-checkmark")))
    time.sleep(1)
    driver.switch_to.default_content()

    # Press the Submit button2
    driver.find_element(By.CSS_SELECTOR, '#submit-btn').click()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
emailtd = driver.find_element(By.CSS_SELECTOR, '#details-container > table > tbody > tr:nth-child(1) > td:nth-child(6)')
print(emailtd.find_element(By.CSS_SELECTOR, '#email').text)

In [14]:
sign_in = driver.find_element(By.CSS_SELECTOR, 'a[class = "yt-simple-endpoint style-scope yt-formatted-string"]')
sign_in.click()

In [15]:
driver.find_element(By.ID, 'identifierId').send_keys(email)

driver.find_element(
    By.CSS_SELECTOR, '#identifierNext > div > button > span').click()

# password_selector = "#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input"

# WebDriverWait(browser, 10).until(
#     EC.visibility_of_element_located((By.CSS_SELECTOR, password_selector)))

# browser.find_element(
#     By.CSS_SELECTOR, password_selector).send_keys(password)

# browser.find_element(
#     By.CSS_SELECTOR, '#passwordNext > div > button > span').click()

In [16]:
cyber_user = driver.find_element(By.NAME, 'username')
cyber_user.send_keys('162012133055')

cyber_pass = driver.find_element(By.NAME, 'passwd')
cyber_pass.send_keys('farras210102')

cyber_signin = driver.find_element(By.NAME, 'signIn')
cyber_signin.click()


In [17]:
cont = driver.find_element(By.CSS_SELECTOR, 'span[class = "VfPpkd-vQzf8d"]')
cont.click()

In [19]:
see_email = driver.find_element(By.CSS_SELECTOR, 'ytd-button-renderer[class = "style-scope ytd-channel-about-metadata-renderer style-default size-default"]')
driver.execute_script("arguments[0].click();", see_email)
